In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
import sys
sys.path.append('/content/yolov5/')

In [ ]:
from yolov5.models.experimental import attempt_load
import torch

model_path='/content/drive/MyDrive/PoolAngel/crowdhuman_yolov5m.pt'

device=torch.device('cpu')
pytorch_model = attempt_load(model_path)

im_size=640

dummy_input = torch.rand((1, 3, im_size, im_size))

onnx_model_path = model_path.replace('pt','onnx')
torch.onnx.export(pytorch_model, dummy_input, onnx_model_path, export_params=True,  # store the trained parameter weights inside the model file
         opset_version=12,    # the ONNX version to export the model to
         do_constant_folding=True,  # whether to execute constant folding for optimization
         input_names = ['modelInput'],   # the model's input names
         output_names = ['modelOutput'], # the model's output names
        )
print('onnx_expoted')

In [ ]:
# Set path variables
onnx_path = model_path.replace('pt','onnx')

# Check that correct files have been uploaded
import os

assert os.path.exists(onnx_path)

print("Files uploaded successfully.")

In [ ]:
# onnx-tf was designed for TensorFlow 1.X, so force this version.
#%tensorflow_version 1.x
import tensorflow as tf

# "!" allows command-line input. Use pip package manager to install the
# conversion package
!pip install onnx-tf

In [ ]:
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load(onnx_path)
tf_rep = prepare(onnx_model)

In [ ]:
pb_path = "crowdhuman_yolov5m.pb"
tf_rep.export_graph(pb_path)

assert os.path.exists(pb_path)
print(".pb model converted successfully.")

In [ ]:
input_nodes = tf_rep.inputs
output_nodes = tf_rep.outputs
print("The names of the input nodes are: {}".format(input_nodes))
print("The names of the output nodes are: {}".format(output_nodes))

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(pb_path) #,
                                                      #input_arrays=input_nodes,
                                                      #output_arrays=output_nodes)
tflite_rep = converter.convert()

In [ ]:
tflite_path = "crowdhuman_yolov5m.tflite"
open(tflite_path, "wb").write(tflite_rep)

assert os.path.exists(tflite_path)
print(".tflite model converted successfully.")

In [ ]:
!pip3 install tflite_runtime

In [ ]:

import tflite_runtime.interpreter as tflite
import numpy as np
import cv2
import matplotlib.pyplot as plt

## Test object detection model  ###

model_file='/content/crowdhuman_yolov5m.tflite'

interpreter = tflite.Interpreter(model_path=model_file)

interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']
image=cv2.imread('/content/drive/MyDrive/PoolAngel/vid2-005.jpg')
image=cv2.resize(image, (640,640))[:,:,::-1].transpose([2,0,1])

input_data = image
input_data=input_data.astype('float32')[None]

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[3]['index'])
pred=output_data[0]

# pred=pred.reshape(-1,5)
print(pred.shape)